In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
import seaborn as sns
from scipy import stats
# Para QQ-plot
import statsmodels.api as sm

# nível de significância de 5% com nível de confiabilidade de 95%
alpha = 0.05 

# leitura do dataset nhames que esta arquivado em um Volume do Unity Catalog
df = pd.read_csv('/Volumes/data_catalog_01_d/bronze/vol_data/nhanes_2015_2016_v2.csv')

### Questão 1 – No dataset NHANES, considere a variável IMC
1. Separe os indivíduos em dois grupos:
Grupo A: indivíduos com idade menor que 40 anos
Grupo B: indivíduos com idade maior ou igual a 40 anos
2. Formule corretamente a hipótese nula (H0) e a hipótese alternativa
(H1) para verificar se existe diferença estatística no IMC
médio entre os grupos.
3. Justifique:
se o teste deve ser bicaudal ou unicaudal
qual critério de rejeição você adotaria (α)

In [0]:
# 1. Separação dos grupos
grupo_A = df[df['Idade'] < 40]['IMC']
grupo_B = df[df['Idade'] >= 40]['IMC']

m_a = grupo_A.mean()
m_b = grupo_B.mean()
print(f'Media do grupo A: {m_a}')
print(f'Media do grupo B: {m_b}')

# 2. Formulação das hipóteses
# H0 (Hipótese nula): Não existe diferença estatística no IMC médio entre os grupos.
#     H0: μ_A = μ_B
# H1 (Hipótese alternativa): Existe diferença estatística no IMC médio entre os grupos.
#     H1: μ_A ≠ μ_B

# Hipótese escolhida: H0 (Hipótese nula)
# Por quê: O teste estatístico parte do pressuposto de que não existe diferença entre os grupos (H0).
# Apenas se os resultados do teste indicarem evidência suficiente contra H0 (p-valor < α), rejeitaremos H0 em favor de H1.

# 3. Justificativa
# O teste deve ser bicaudal porque estamos interessados em detectar qualquer diferença significativa no IMC médio entre os grupos,
# seja para mais ou para menos. Mesmo que a média do grupo B seja maior (29 vs 28), o teste bicaudal avalia a possibilidade
# de diferença em ambas as direções, sem assumir previamente qual grupo teria média maior. Isso garante uma análise imparcial
# e adequada para hipóteses de igualdade (H0: μ_A = μ_B) versus diferença (H1: μ_A ≠ μ_B).
# O critério de rejeição adotado será α = 0.05 (nível de significância de 5%).


# Gráfico para comparar a distribuição do IMC entre os grupos
plt.figure(figsize=(8, 5))
sns.histplot(grupo_A, color='blue', label='Idade < 40', kde=True, stat='density', alpha=0.5)
sns.histplot(grupo_B, color='red', label='Idade >= 40', kde=True, stat='density', alpha=0.5)
plt.xlabel('IMC')
plt.ylabel('Densidade')
plt.title('Distribuição do IMC por Grupo de Idade')
plt.legend()
plt.tight_layout()
plt.show()

In [0]:
# O teste deve ser bicaudal porque estamos interessados em detectar qualquer diferença significativa no IMC médio entre os grupos,
# seja para mais ou para menos. Mesmo que a média do grupo B seja maior (29 vs 28), o teste bicaudal avalia a possibilidade
# de diferença em ambas as direções, sem assumir previamente qual grupo teria média maior. Isso garante uma análise imparcial
# e adequada para hipóteses de igualdade (H0: μ_A = μ_B) versus diferença (H1: μ_A ≠ μ_B).

### Questão 2 - Considere a variável Sexo e a variável Peso.
1. Avalie, conceitualmente, se o teste t independente seria adequado.
para comparar o peso entre homens e mulheres.
2. Caso a suposição de normalidade não seja válida, indique:
qual teste estatístico alternativo deve ser utilizado, e
por que ele é mais apropriado neste contexto.
3. Explique o que exatamente o teste escolhido está comparando, à luz do conteúdo teórico da aula.

In [0]:
# 1. Avalie, conceitualmente, se o teste t independente seria adequado para comparar o peso entre homens e mulheres.
#
# O teste t independente é conceitualmente apropriado para comparar o peso médio entre homens e mulheres, desde que as 
# premissas de normalidade e homogeneidade das variâncias sejam atendidas. Este teste avalia se o peso médio difere 
# significativamente entre os dois grupos independentes (homens vs. mulheres).



# 2. Caso a suposição de normalidade não seja válida, indique: qual teste estatístico alternativo deve ser utilizado, 
# e por que ele é mais apropriado neste contexto.
#
# Se a premissa de normalidade não for válida, deve-se utilizar o teste U de Mann-Whitney (também conhecido como teste de 
# soma de postos de Wilcoxon). Este teste não paramétrico não exige que os dados sigam uma distribuição normal e é mais 
# apropriado para comparar as distribuições de peso entre dois grupos independentes quando a normalidade é violada.



# 3. Explique o que exatamente o teste escolhido está comparando, à luz do conteúdo teórico da aula.
# 
# O teste U de Mann-Whitney compara as posições dos pesos entre homens e mulheres, avaliando se um grupo tende a 
# apresentar valores maiores ou menores que o outro. Ele testa a hipótese nula de que as distribuições de peso em 
# ambos os grupos são iguais, focando nas diferenças de tendência central (medianas) em vez das médias.

# Demonstração


peso_homens = df[df['Sexo']==1]['Peso_kg'].dropna()
peso_mulheres = df[df['Sexo']==2]['Peso_kg'].dropna()

# Teste de normalidade (Shapiro-Wilk)
shapiro_homens = stats.shapiro(peso_homens)
shapiro_mulheres = stats.shapiro(peso_mulheres)

print(f'Shapiro-Wilk Homens: estatística={shapiro_homens.statistic:.4f}, p-valor={shapiro_homens.pvalue:.4f}')
print(f'Shapiro-Wilk Mulheres: estatística={shapiro_mulheres.statistic:.4f}, p-valor={shapiro_mulheres.pvalue:.4f}')

# Se p-valor < 0.05, rejeita normalidade. Caso não seja normal, aplica Mann-Whitney
mannwhitney = stats.mannwhitneyu(peso_homens, peso_mulheres, alternative='two-sided')
print(f'Mann-Whitney U: estatística={mannwhitney.statistic:.4f}, p-valor={mannwhitney.pvalue:.4f}')

# Visualização das distribuições
plt.figure(figsize=(8, 5))
sns.histplot(peso_homens, color='blue', label='Homens', kde=True, stat='density', alpha=0.5)
sns.histplot(peso_mulheres, color='red', label='Mulheres', kde=True, stat='density', alpha=0.5)
plt.xlabel('Peso')
plt.ylabel('Densidade')
plt.title('Distribuição do Peso por Sexo')
plt.legend()
plt.tight_layout()
plt.show()

### Questão 3 – Teste ANOVA e interpretação estatística
Utilize a variável "grupo étnico" como variável categórica e IMC como variável numérica.
Justifique o uso de uma One-Way ANOVA nesse problema.

Explique:
- O significado estatístico do F-test
- O papel do valor-p na decisão

Suponha que o teste indique significância estatística:
- O que exatamente você pode concluir?
- O que não pode ser concluído apenas com o resultado da ANOVA?

In [0]:
# A One-Way ANOVA é adequada neste contexto porque queremos comparar as médias do IMC entre três ou mais grupos independentes 
# definidos pela variável categórica "grupo étnico". O teste ANOVA avalia se existe diferença estatisticamente significativa 
# entre as médias dos grupos, considerando a variabilidade interna e entre grupos.

# Significado estatístico do F-test: 
# O F-test compara a variância entre as médias dos grupos com a variância dentro dos grupos. Um valor F alto sugere que as 
# diferenças entre as médias dos grupos são maiores do que seria esperado pelo acaso, indicando possível diferença real entre os 
# grupos.

# Papel do valor-p na decisão: 
# O valor-p indica a probabilidade de observar uma diferença entre as médias dos grupos tão grande quanto a encontrada (ou maior), 
# asumindo que a hipótese nula (todas as médias são iguais) seja verdadeira. Se o valor-p for menor que o nível de significância 
# (por exemplo, 0,05), rejeitamos a hipótese nula e concluímos que há diferença estatística entre pelo menos dois grupos.

# Se o teste indicar significância estatística: 
# Você pode concluir que existe pelo menos uma diferença estatisticamente significativa entre as médias de IMC dos grupos étnicos 
# analisados.

# O que não pode ser concluído apenas com o resultado da ANOVA: 
# A ANOVA não informa quais grupos são diferentes entre si, nem a direção ou magnitude dessas diferenças. Para identificar quais 
# pares de grupos apresentam diferenças significativas, é necessário realizar testes post-hoc (como Tukey, Bonferroni, etc.).

### Questão 4 – Correlação, monotonicidade(rank) e escolha do coeficiente

Considere as variáveis Altura e Peso.
1. Calcule conceitualmente (sem código) em que situações a correlação de Pearson pode ser inadequada.
2. Explique quando e por que a correlação de Spearman seria mais apropriada para esses dados.
3. Descreva um cenário realista no NHANES em que:
Pearson ≈ 0 mas Spearman indique correlação estatisticamente significativa.

In [0]:
# A correlação de Pearson pode ser inadequada quando:
# - A relação entre Altura e Peso não é linear (por exemplo, curvilínea).
# - Os dados apresentam outliers extremos que distorcem a média e a variância.
# - As variáveis não seguem distribuição normal.
# - Existem heterocedasticidade (variância não constante) ou escalas diferentes.

# A correlação de Spearman é mais apropriada quando:
# - A relação entre Altura e Peso é monotônica, mas não necessariamente linear.
# - Os dados possuem outliers ou não são normalmente distribuídos.
# - Queremos avaliar se o ranking (ordem) de Altura acompanha o ranking de Peso, independentemente da forma da relação.

# Cenário realista no NHANES:
# Suponha que, em uma amostra, pessoas com altura média tenham grande variação de peso (atletas, pessoas com sobrepeso, etc.),
# enquanto pessoas muito baixas ou muito altas tendem a ter pesos mais extremos (menores ou maiores). 
# Isso pode gerar uma relação não linear: para alturas médias, o peso varia muito, mas para extremos de altura, o peso acompanha.
# Nesse caso, Pearson ≈ 0 (não detecta relação linear), mas Spearman pode indicar correlação significativa, pois os rankings
# de altura e peso ainda acompanham uma tendência monotônica (quanto maior a altura, maior o peso em geral, mesmo sem linearidade).

### Questão 5 – Estatística inferencial da Regressão Linear
Considere um modelo qualquer de regressão linear múltipla para
explicar IMC, usando como variáveis independentes:
- Idade
- Altura
- Peso

1. Explique a diferença entre:
- teste t individual dos coeficientes
- teste F global

2. Se o teste F global rejeita H0, mas apenas uma variável apresenta p-valor individual significativo, como esse resultado deve ser interpretado?

3. Diga qual foi a implicação disso para:
- multicolinearidade (faça análise das correlações)
- matriz de correlação
- interpretação do R²

A resposta deve ser analítica e estatística, não computacional.

In [0]:
# Diferença entre teste t individual dos coeficientes e teste F global:
# - O teste t individual avalia se cada coeficiente (Idade, Altura, Peso) é estatisticamente diferente de zero, ou seja, se cada variável tem efeito significativo sobre o IMC, controlando pelas demais.
# - O teste F global avalia se, considerando todas as variáveis juntas, o modelo explica uma fração significativa da variabilidade do IMC (ou seja, se pelo menos um coeficiente é diferente de zero).

# Interpretação quando o teste F global rejeita H0, mas apenas uma variável tem p-valor individual significativo:
# - O modelo como um todo é significativo (há associação entre IMC e pelo menos uma das variáveis), mas apenas uma variável contribui individualmente de forma estatisticamente significativa. As demais podem não ter efeito próprio detectável, possivelmente devido à multicolinearidade ou baixa variabilidade.

# Implicações para multicolinearidade:
# - Multicolinearidade ocorre quando variáveis independentes são altamente correlacionadas entre si, dificultando a separação dos efeitos individuais. Isso pode levar a p-valores altos nos testes t individuais, mesmo que o modelo global seja significativo.
# - Para analisar, deve-se examinar a matriz de correlação entre Idade, Altura e Peso. Correlações altas (>0.7 ou <-0.7) sugerem multicolinearidade.

# Matriz de correlação:
# - A matriz de correlação mostra o grau de associação linear entre as variáveis independentes. Correlações elevadas indicam risco de multicolinearidade, o que pode inflar erros padrão dos coeficientes e dificultar a interpretação dos efeitos individuais.

# Interpretação do R²:
# - O R² indica a proporção da variância do IMC explicada pelo modelo. Mesmo com multicolinearidade, o R² pode ser alto, pois o modelo como um todo ajusta bem os dados, mas os efeitos individuais das variáveis podem ser imprecisos.
# - Portanto, um R² alto com poucos coeficientes significativos sugere que as variáveis juntas explicam o IMC, mas é difícil atribuir o efeito a uma variável específica devido à sobreposição de informações (multicolinearidade).